In [ ]:
import os
os.mkdir('/content/test')
os.mkdir('/content/train')

In [ ]:
!cp '/content/drive/MyDrive/AI_Project/test.zip' '/content/test'

In [ ]:
!cp '/content/drive/MyDrive/AI_Project/train.zip' '/content/train'

In [ ]:
import os
os.chdir('/content/test')
!unzip '/content/test/test.zip'

Streaming output truncated to the last 5000 lines.
  inflating: face04688.jpg           
  inflating: face03972.jpg           
  inflating: face02593.jpg           
  inflating: face00836.jpg           
  inflating: face05284.jpg           
  inflating: face03261.jpg           
  inflating: face00582.jpg           
  inflating: face02696.jpg           
  inflating: face05325.jpg           
  inflating: face04973.jpg           
  inflating: face02902.jpg           
  inflating: face06245.jpg           
  inflating: face00645.jpg           
  inflating: face03284.jpg           
  inflating: face00188.jpg           
  inflating: face05490.jpg           
  inflating: face01683.jpg           
  inflating: face05566.jpg           
  inflating: face03288.jpg           
  inflating: face00045.jpg           
  inflating: face06050.jpg           
  inflating: face02871.jpg           
  inflating: face04507.jpg           
  inflating: face01597.jpg           
  inflating: face03175.jpg           

In [ ]:
import os
os.chdir('/content/train')
!unzip '/content/train/train.zip'

Streaming output truncated to the last 5000 lines.
  inflating: F0099/MID1/P11523_face3.jpg  
  inflating: F0099/MID1/P11536_face2.jpg  
  inflating: F0099/MID1/P01025_face1.jpg  
  inflating: F0099/MID1/P11539_face1.jpg  
  inflating: F0099/MID1/P01021_face1.jpg  
  inflating: F0099/MID1/P01026_face1.jpg  
  inflating: F0099/MID1/P11519_face3.jpg  
  inflating: F0099/MID1/P11535_face2.jpg  
  inflating: F0099/MID1/P01027_face1.jpg  
  inflating: F0099/MID1/P11518_face2.jpg  
  inflating: F0099/MID1/P11520_face3.jpg  
  inflating: F0099/MID6/P11523_face2.jpg  
  inflating: F0099/MID6/P11518_face4.jpg  
  inflating: F0099/MID6/P11522_face4.jpg  
  inflating: F0099/MID6/P11519_face2.jpg  
  inflating: F0099/MID6/P11539_face2.jpg  
  inflating: F0099/MID3/P01019_face3.jpg  
  inflating: F0099/MID3/P01021_face3.jpg  
  inflating: F0099/MID3/P11518_face3.jpg  
  inflating: F0099/MID3/P11520_face2.jpg  
  inflating: F0099/MID3/P11519_face5.jpg  
  inflating: F0099/MID3/P01020_face3.jpg  
   

In [ ]:
import os
os.chdir('/content')

In [ ]:
!rm '/content/train/train.zip'
!rm '/content/test/test.zip'

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-81ig0kax
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-81ig0kax
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8325 sha256=230f28bd0c8b83ab59689b2c3931560a867887aff79b5a450794cdc799a2da05
  Stored in directory: /tmp/pip-ephem-wheel-cache-xabunc0t/wheels/08/df/86/0225d44647ab2256dbf1e006823288fe9cc86367a056e6ea2c
Successfully built keras-vggface


In [ ]:
!pip install keras_applications

     |████████████████████████████████| 50 kB 3.0 MB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from collections import defaultdict
from glob import glob
from random import choice, sample
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Reshape, Flatten
from keras.models import Model
from keras.optimizers import Adam, Adagrad, RMSprop
from keras import regularizers
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [ ]:
train_file_path = "train_relationships.csv"
train_folders_path = "train/"
val_famillies = "F01"

all_images = glob(train_folders_path + "*/*/*.jpg")

train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [ ]:
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

# Model 1

In [ ]:
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers:
        x.trainable = False

    for x in base_model.layers[-3:]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1=Concatenate(axis=-1)([GlobalMaxPool2D()(x1),GlobalAvgPool2D()(x1)])
    x2=Concatenate(axis=-1)([GlobalMaxPool2D()(x2),GlobalAvgPool2D()(x2)])

    x3=Subtract()([x1,x2])
    x3=Multiply()([x3,x3])
    
    x1_=Multiply()([x1,x1])
    x2_=Multiply()([x2,x2])
    x4=Subtract()([x1_,x2_])

    x5=Multiply()([x1,x2])
    
    x=Concatenate(axis=-1)([x3,x4,x5])
    
    x=Dense(100,activation='relu')(x)
    x=Dropout(0.01)(x)
    out=Dense(1,activation='sigmoid')(x)

    model=Model([input_1,input_2],out)

    model.compile(loss='binary_crossentropy',metrics=['acc'],optimizer=Adam(0.00001))

    model.summary()

    #x = Concatenate()([x1, x2])
    #x = Flatten()(x)
    #x = Dense(1024, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.4)(x)
    #out = Dense(1, activation="sigmoid")(x)

    #model = Model([input_1, input_2], out)

    #model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=RMSprop(lr=1e-4))

    #model.summary()

    return model

In [ ]:
file_path = "vgg_face.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

es = EarlyStopping(monitor='val_acc',patience=20)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()
#model.load_weights(file_path)
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=400, validation_steps=100)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_3 (GlobalM (None, 2048)         0           vggface_resnet50[1][0]     

/tensorflow-1.15.2/python3.7/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/50
 - 124s - loss: 4.2548 - acc: 0.5658 - val_loss: 7.6217 - val_acc: 0.6037

Epoch 00001: val_acc improved from -inf to 0.60375, saving model to vgg_face.h5
Epoch 2/50
 - 120s - loss: 2.7630 - acc: 0.6339 - val_loss: 3.6661 - val_acc: 0.6444

Epoch 00002: val_acc improved from 0.60375 to 0.64438, saving model to vgg_face.h5
Epoch 3/50
 - 120s - loss: 2.1973 - acc: 0.6736 - val_loss: 4.1467 - val_acc: 0.6506

Epoch 00003: val_acc improved from 0.64438 to 0.65062, saving model to vgg_face.h5
Epoch 4/50
 - 120s - loss: 1.9210 - acc: 0.6705 - val_loss: 2.5956 - val_acc: 0.6700

Epoch 00004: val_acc improved from 0.65062 to 0.67000, saving model to vgg_face.h5
Epoch 5/50
 - 120s - loss: 1.5717 - acc: 0.6878 - val_loss: 3.5057 - val_acc: 0.6669

Epoch 00005: val_acc did not improve from 0.67000
Epoch 6/50
 - 121s - loss: 1.4007 - acc: 0.6963 - val_loss: 1.5977 - val_acc: 0.6581

Epoch 00006: val_acc did not improve from 0.67000
Epoch 7/50
 - 121s - loss: 1.2306 - acc: 0.6998 - val_l

In [ ]:
!cp '/content/vgg_face.h5' '/content/drive/MyDrive/AI_Project/Models'

In [ ]:
model = baseline_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
______________

In [ ]:
test_path = "test/"
model.load_weights('vgg_face.h5')  

def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv('sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("vgg_face.csv", index=False)

0it [00:00, ?it/s]

166it [01:56,  1.43it/s]


In [ ]:
!cp '/content/vgg_face.csv' '/content/drive/MyDrive/AI_Project/Models'

# Model 2

In [ ]:
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers:
        x.trainable = False

    for x in base_model.layers[-5:]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1=Concatenate(axis=-1)([GlobalMaxPool2D()(x1),GlobalAvgPool2D()(x1)])
    x2=Concatenate(axis=-1)([GlobalMaxPool2D()(x2),GlobalAvgPool2D()(x2)])

    x3=Subtract()([x1,x2])
    x3=Multiply()([x3,x3])
    
    x1_=Multiply()([x1,x1])
    x2_=Multiply()([x2,x2])
    x4=Subtract()([x1_,x2_])

    x5=Multiply()([x1,x2])
    
    x=Concatenate(axis=-1)([x3,x4,x5])
    
    x=Dense(100,activation='relu')(x)
    x=Dropout(0.01)(x)
    out=Dense(1,activation='sigmoid')(x)

    model=Model([input_1,input_2],out)

    model.compile(loss='binary_crossentropy',metrics=['acc'],optimizer=Adam(0.00001))

    model.summary()

    #x = Concatenate()([x1, x2])
    #x = Flatten()(x)
    #x = Dense(1024, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.4)(x)
    #out = Dense(1, activation="sigmoid")(x)

    #model = Model([input_1, input_2], out)

    #model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=RMSprop(lr=1e-4))

    #model.summary()

    return model

In [ ]:
file_path = "vgg_face2.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

es = EarlyStopping(monitor='val_acc',patience=20)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()
#model.load_weights(file_path)
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=1,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=400, validation_steps=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


94699520/94694792 [==============================] - 1s 0us/step


AttributeError: ignored

# Model 3

In [ ]:
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers:
        x.trainable = False

    for x in base_model.layers[-5::]:
        x.trainable = True
    
    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1=Concatenate(axis=-1)([GlobalMaxPool2D()(x1),GlobalAvgPool2D()(x1)])
    x2=Concatenate(axis=-1)([GlobalMaxPool2D()(x2),GlobalAvgPool2D()(x2)])

    x3=Subtract()([x1,x2])
    x3=Multiply()([x3,x3])
    
    x1_=Multiply()([x1,x1])
    x2_=Multiply()([x2,x2])
    x4=Subtract()([x1_,x2_])

    x5=Multiply()([x1,x2])
    
    x=Concatenate(axis=-1)([x3,x4,x5])

    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x) 
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x)    
    x   = Dense(100, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)    
    out = Dense(1, activation="sigmoid")(x)
  
    model=Model([input_1,input_2],out)

    model.compile(loss='binary_crossentropy',metrics=['acc'],optimizer=Adam(0.00001))

    model.summary()

    #x = Concatenate()([x1, x2])
    #x = Flatten()(x)
    #x = Dense(1024, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.5)(x)
    #x = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    #x = Dropout(0.4)(x)
    #out = Dense(1, activation="sigmoid")(x)

    #model = Model([input_1, input_2], out)

    #model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=RMSprop(lr=1e-4))

    #model.summary()

    return model

In [ ]:
model = baseline_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
______________

In [ ]:
file_path = "vgg_face3.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

es = EarlyStopping(monitor='val_acc',patience=20)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()
#model.load_weights(file_path)
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), max_queue_size=10,                
                                  workers=4,                        
                                  use_multiprocessing=True,   
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=100, verbose=1, callbacks=callbacks_list, steps_per_epoch=400, validation_steps=100)

KeyboardInterrupt: ignored

In [ ]:
!cp '/content/vgg_face3.h5' '/content/drive/MyDrive/AI_Project/Models'

In [ ]:
import pickle
with open('history.pkl','wb') as f:
  pickle.dump(history,f)

In [ ]:
test_path = "test/"
model.load_weights('vgg_face3.h5')  

def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv('sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("vgg_face.csv", index=False)

0it [00:00, ?it/s]

166it [01:56,  1.42it/s]


In [ ]:
!cp '/content/vgg_face3.csv' '/content/drive/MyDrive/AI_Project/Models'